In [0]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta



In [0]:
filename = "group_organized_events"
json_file_path = f"silver/{filename}.json"

In [0]:
# Calculate the date 90 days ago
start_date = datetime.now() - timedelta(days=90)

# Extract month and year from the calculated date
month_num = start_date.month
year = start_date.year

# Construct the URL using f-string
url = f'https://steamcommunity.com/groups/higps/events?xml=1&action=eventFeed&month={month_num}&year={year}'




In [0]:
def get_event_dict(url):
    html_strings  = []
    events_list = []

    # Fetch the XML content from the URL
    response = requests.get(url)
    xml_content = response.content

    # Parse the XML content
    tree = ET.fromstring(xml_content)

    # Access elements and attributes
    for expired_event in tree.iter('expiredEvent'):
        event_id = expired_event.get('eventID')
        event_data = expired_event.text  # Contains the CDATA content
        html_strings .append(event_data)
        # Your processing logic here

    for html_string in html_strings:
        # Parse the HTML string with BeautifulSoup
        soup = BeautifulSoup(html_string, 'html.parser')

        # Extract relevant information
        event_id = soup.find('div', class_='eventBlock')['id']
        weekday = soup.find('span', class_='hiliteTextRed').text.strip()
        event_time = soup.find('span', class_='eventDateTime').text.strip()
        event_title = soup.find('a', class_='headlineLink').text.strip()
        event_link = soup.find('a', class_='headlineLink')['href']

        # Create a dictionary for the current event
        event_dict = {
            'eventID': event_id,
            'weekday': weekday,
            'eventTime': event_time,
            'eventTitle': event_title,
            'eventLink': event_link,
        }

        # Append the dictionary to the list
        events_list.append(event_dict)

    return events_list

In [0]:

 # Convert events_list to a DataFrame
df = pd.DataFrame(get_event_dict(url))

# Print the DataFrame
print(df.head())

In [0]:
dfs = []
dfs = []
num_days = 90

today = datetime.today()
# Calculate the date 90 days ago from today
start_date = today - timedelta(days=num_days)
print(start_date)

# Initialize variables for the first iteration
current_month = 0
current_year = 0

# Loop through each day for the specified number of days
for _ in range(num_days):
    # Calculate the current date
    current_date = start_date + timedelta(days=1)

    # Check if the month and year have changed
    if current_date.month != current_month or current_date.year != current_year:
        # If the month and year have changed, update the variables
        current_month = current_date.month
        current_year = current_date.year

        # Construct the URL using f-string with month and year
        url = f'https://steamcommunity.com/groups/higps/events?xml=1&action=eventFeed&month={current_month}&year={current_year}'

        # Fetch data from the URL and process it
        events_list = get_event_dict(url)

        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(events_list)
        df['month'] = current_month
        df['year'] = current_year
        # Append the DataFrame to the list
        dfs.append(df)

    # Update the start date for the next iteration
    start_date = current_date

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

In [0]:
#"eventTime":"2024-01-01 11:43:06"

new_df = final_df.copy()


In [0]:

# Split the 'Weekday' column into two columns: 'Weekday' and 'WeekdayNum'
new_df[['weekday', 'WeekdayNum']] = new_df['weekday'].str.extract(r'([a-zA-Z]+) (\d*)', expand=True)

# Convert 'WeekdayNum' to numeric (integer), treating empty strings as NaN
new_df['WeekdayNum'] = pd.to_numeric(new_df['WeekdayNum'], errors='coerce', downcast='integer')



In [0]:
# Display the updated DataFrame
reduced_df = new_df[["weekday", "eventTime", "eventTitle","month", "year", "WeekdayNum"]]

In [0]:
reduced_df

In [0]:
reduced_df['combined_eventTime'] = (
    reduced_df['year'].astype(str) + '-' +
    reduced_df['month'].astype(str).str.zfill(2) + '-' +
    reduced_df['WeekdayNum'].astype(str).str.zfill(2) + ' ' +
    reduced_df['eventTime'] + ':00'
)

In [0]:
def convert_to_24_hour_format(time_str):
    dt_obj = datetime.strptime(time_str, '%I:%M%p')
    return dt_obj.strftime('%H:%M:%S')

In [0]:
reduced_df['combined_eventTime_fixed'] = (
    reduced_df['year'].astype(str) + '-' +
    reduced_df['month'].astype(str).str.zfill(2) + '-' +
    reduced_df['WeekdayNum'].astype(str).str.zfill(2) + ' ' +
    reduced_df['eventTime'].apply(convert_to_24_hour_format)
)

In [0]:
#me connected on the 17th at "playerId":2,"eventTime":"2024-01-17 10:07:30","serverId":11,
# 2024-01-17 10:07:00:00
#local time: 19:07

reduced_df

In [0]:
import pytz

In [0]:
reduced_df = reduced_df[['eventTitle', 'combined_eventTime_fixed', 'weekday']]

# Rename the 'combined_eventTime_fixed' column to 'eventTime'
reduced_df.rename(columns={'combined_eventTime_fixed': 'eventTimeStart'}, inplace=True)

# Display the updated DataFrame
print(reduced_df)

In [0]:
#Snippet to change the timezone of the eventTime
# # Assuming your 'eventTime' column is in string format, convert it to datetime
# reduced_df['eventTimeStart'] = pd.to_datetime(reduced_df['eventTimeStart'])

# # Define time zones
# us_timezone = pytz.timezone('America/Los_Angeles')  # Adjust this based on the actual timezone
# norway_timezone = pytz.timezone('Europe/Oslo')

# # Convert time to Norway timezone
# reduced_df['eventTime_norway'] = reduced_df['eventTimeStart'].dt.tz_localize(us_timezone).dt.tz_convert(norway_timezone)

# # Format the time in 24-hour format
# reduced_df['eventTime_norway'] = reduced_df['eventTime_norway'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [0]:
sparkdf = spark.createDataFrame(reduced_df)

sparkdf.write.mode('overwrite').saveAsTable("adb_hibak.bronze_tf2_"+filename)

In [0]:
sparkdf.display()